# How Time Works

In the graph there is a node labled `time` with no edges. At the beginning `currentTime=0`, and is autoincremented by 1 indefinitely. Time is always running and never reset.

An Azure function `functions/time` runs all events and actions that require time to complete. That runs on an NCRONTAB job specified in the `function.json` doc. It can be adjusted to speed up or slow down time in the game. 

Example NCRONTAB configurations:
* `"0 */2 * * * *"`  Every two minutes
* `"0 */5 * * * *"`  Every five minutes

Note that some events are taken care of by `functions/cosmicevents` and not by time.


In [1]:
import sys
import pandas as pd
sys.path.append('..')

from app.connectors import cmdb_graph
from app.functions import maths
from app.objects import time

c = cmdb_graph.CosmosdbClient()


import  ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

executing local windows deployment


In [2]:
time = time.Time(c)
time

< time at: 2023-09-04T19:40:59.606725+00:00 UTU:None >

The time object doesn't query the graph until it is initialized. 

In [3]:
time.get_current_UTU()
time

< time at: 2023-09-04T19:40:59.606725+00:00 UTU:1000 >

The process just takes the UTU, adds 1 and patches. 

In [4]:
time.global_ticker()

'currentTime was updated from:1000 to: 1001'

You can confirm that it was updated by re-running the `get_current_UTU`. This does not cause an update to the graph, but it does update the UTU in the time object. 

In [5]:
time.get_current_UTU()
time

< time at: 2023-09-04T19:40:59.606725+00:00 UTU:1001 >

In [ ]:
Arbitrary break to keep from auto-running

# Setting time

In [6]:
time_query = f"g.addV('time').property('currentTime',1000).property('updated_at','{time.utc_timestamp}').property('label','time').property('objid','{maths.uuid()}')"


In [7]:
c.run_query(time_query)

## Validate what was created

In [8]:
c.run_query('g.V().hasLabel("time").valueMap()')
c.res

[{'currentTime': [1000],
  'updated_at': ['2023-09-04T19:39:57.653661+00:00'],
  'objid': ['3297686412899']}]

In [5]:
c.run_query('g.V().hasLabel("time").drop()')
c.res

[]